In [2]:
from model import Model
import pandas as pd
import numpy as np
import shap

ModuleNotFoundError: No module named 'model'

In [ ]:

#TODO: доделать _update_score(какая то формула из агрессивности всех поездок, средних звездочек), получить среднюю оценку из новой даты, сделать отображение всего рейтинга

class Driver:
    def __init__(self, id, main_df, comment_df, tracks_df, model):
        '''
        id - driver id
        initial_orders_data - all the driver's orders
        model - catboost model for agressivness prediction
        '''
        
        self.id: str= id
        # хз как устроить лучше
        self.features = {'agressivness':0.0, 'average_mark':0.0, 'speed_limit_ignorance':0.0 }

        self.main_df = main_df
        self.comment_df = comment_df
        self.tracks_df = tracks_df

        for order_id in self.orders_data['order_id'].unique():
            order_data = self.orders_data[['order_id'==order_id]]
            comment_data= self.comment_df[['order_id'==order_id]]
            tracks_data = self.tracks_df[['order_id'==order_id]]
            self.add_order(order_data, comment_data, tracks_data)

        self.model = Model
        self.score = 0.0

    def _update_score(self,):
        ...

    def add_order(self, main_order_data: pd.DataFrame,comment_order_data: pd.DataFrame,track_order_data: pd.DataFrame):
        '''extracts features from order's data 
           and appends it to self.features
        '''
        # возможно стоит сделать усредненный скор по заказу(из звездочек, аргессивности и тд), добавлять к общему и считать среднее(на пeрвое время)
        transformed, order_aggressivness = self.model.predict_proba(main_order_data, comment_order_data, track_order_data)

        shap.initjs()
        shap_values = self.model.explainer(transformed)
        shap.plots.waterfall(shap_values)

In [ ]:
class Rating:
    def __init__(self, initial_df=None, initial_comment_df=None, initial_tracks_df=None):
        self.drivers = []
        self.main_df = initial_df
        self.comment_df = initial_comment_df
        self.tracks_df = initial_tracks_df
        self.model = Model() 

    def initial(self):
        for driver_id in self.main_df['driver_id'].unique():

            # append a driver with all his orders from initial data to the driver list 
            driver = Driver(driver_id, self.main_df[['driver_id'==driver_id]],self.comment_df[['driver_id'==driver_id]],self.tracks_df[['driver_id'==driver_id]], self.model)
            self.drivers.append(driver)
    
    def output_rating(self):
        ...
